In [21]:
import pandas as pd

df = pd.read_csv("sqf-2023.csv", na_values="(null)", low_memory=False)

threshold = 0.4 * len(df)

columns_to_remove = df.columns[df.isnull().sum() > threshold]

df_cleaned = df.loc[:, df.isnull().sum() <= threshold]

df_cleaned.to_csv("sqf-2023-cleaned.csv", index=False)

# Print the results
print(f"columns removed({len(columns_to_remove)}):")
print(columns_to_remove.tolist())

print(len(df.columns))
print(len(df_cleaned.columns))
print(df_cleaned.columns.tolist())

columns removed(36):
['JURISDICTION_CODE', 'JURISDICTION_DESCRIPTION', 'OFFICER_NOT_EXPLAINED_STOP_DESCRIPTION', 'SUSPECT_ARREST_OFFENSE', 'SUMMONS_OFFENSE_DESCRIPTION', 'ID_CARD_IDENTIFIES_OFFICER_FLAG', 'SHIELD_IDENTIFIES_OFFICER_FLAG', 'VERBAL_IDENTIFIES_OFFICER_FLAG', 'FIREARM_FLAG', 'KNIFE_CUTTER_FLAG', 'OTHER_WEAPON_FLAG', 'PHYSICAL_FORCE_CEW_FLAG', 'PHYSICAL_FORCE_DRAW_POINT_FIREARM_FLAG', 'PHYSICAL_FORCE_HANDCUFF_SUSPECT_FLAG', 'PHYSICAL_FORCE_OC_SPRAY_USED_FLAG', 'PHYSICAL_FORCE_OTHER_FLAG', 'PHYSICAL_FORCE_RESTRAINT_USED_FLAG', 'PHYSICAL_FORCE_WEAPON_IMPACT_FLAG', 'BACKROUND_CIRCUMSTANCES_VIOLENT_CRIME_FLAG', 'BACKROUND_CIRCUMSTANCES_SUSPECT_KNOWN_TO_CARRY_WEAPON_FLAG', 'SUSPECTS_ACTIONS_CASING_FLAG', 'SUSPECTS_ACTIONS_CONCEALED_POSSESSION_WEAPON_FLAG', 'SUSPECTS_ACTIONS_DECRIPTION_FLAG', 'SUSPECTS_ACTIONS_DRUG_TRANSACTIONS_FLAG', 'SUSPECTS_ACTIONS_IDENTIFY_CRIME_PATTERN_FLAG', 'SUSPECTS_ACTIONS_LOOKOUT_FLAG', 'SUSPECTS_ACTIONS_OTHER_FLAG', 'SUSPECTS_ACTIONS_PROXIMITY_TO_SCEN

In [22]:
import pandas as pd

df_cleaned = pd.read_csv("sqf-2023-cleaned.csv", na_values=["(null)", ""], low_memory=False)

for col in df_cleaned.columns:
    if pd.api.types.is_numeric_dtype(df_cleaned[col]):
        # mean for numerical columns
        mean_value = df_cleaned[col].mean()
        df_cleaned[col].fillna(mean_value, inplace=True)
    else:
        # mode for categorical columns
        mode = df_cleaned[col].mode()
        if not mode.empty:
            df_cleaned[col].fillna(mode[0], inplace=True)

df_cleaned.to_csv("sqf-2023-cleaned.csv", index=False)

/var/folders/my/mt8npr5n1tx6kxqgbd6_81km0000gn/T/ipykernel_33052/477796308.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[col].fillna(mean_value, inplace=True)
/var/folders/my/mt8npr5n1tx6kxqgbd6_81km0000gn/T/ipykernel_33052/477796308.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [23]:

df = pd.read_csv("sqf-2023-cleaned.csv", na_values=["(null)", ""], low_memory=False)

total_null_or_empty = df.isnull().sum().sum()

print(f"Total '(null)' or empty cells: {total_null_or_empty}")

Total '(null)' or empty cells: 0
